In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the emotion detection model
emotion_model_path = 'D:/PROJECT/emotion_detection_model.keras'

try:
    emotion_model = load_model(emotion_model_path)
    print("Emotion model loaded successfully.")
except Exception as e:
    print(f"Error loading emotion model: {e}")

# Emotion labels for the FER-2013 dataset
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

def predict_emotion(face_image):
    face_image = cv2.resize(face_image, (48, 48))
    face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
    face_image = face_image.astype('float32') / 255.0
    face_image = np.expand_dims(face_image, axis=(0, -1))
    
    prediction = emotion_model.predict(face_image)
    emotion = emotion_labels[np.argmax(prediction)]
    
    return emotion

def detect_and_display(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        face_image = frame[y:y+h, x:x+w]
        emotion = predict_emotion(face_image)
        
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        # Display the label
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    
    return frame

def main():
    cap = cv2.VideoCapture(0)  # Start the webcam
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = detect_and_display(frame)
        cv2.imshow('Emotion Detection', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
